# Подготовка данных для предсказания и замены чисел вместо пропусков

## Импорт библиотек

In [30]:
!pip install miceforest

In [31]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import miceforest as mf
from sklearn.base import clone
from itertools import combinations
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

##Функции/Классы

### **функции**

In [32]:
def nan_to_median(series: pd.Series):
    """
    получает pd.Series с пропусками.
    возвращает pd.Series с медианой вместо пропусков.
    """
    median_val = series.median()
    return series.fillna(median_val)


####################################################################################################


def radical_iqr_filter(df: pd.DataFrame, column: str, lower_bound=True, upper_bound=True, fill_nan=False, drop_nan_values=False, multp=3):
    """
    гибкая функция замены выбросов на NaN с помощью настраиваемого интерквартильного размаха

    Аргументы
        df: DataFrame содержащий данные, которые нужно отфильтровать.

        column: Название столбца в DataFrame df, в которомом будет производиться фильтрация.
        Функция будет рассчитывать IQR именно для этого столбца.

        lower_bound:  Булевый флаг, определяющий, следует ли отфильтровывать значения,
        лежащие ниже нижней границы, рассчитанной на основе IQR. То есть, если
        lower_bound = False, то все выбросы(если они есть) будут игнорироваться.

        upper_bound: Булевый флаг, определяющий, следует ли отфильтровывать значения,
        лежащие выше верхней границы, рассчитанной на основе IQR.

        multp: Множитель, используемый для расчета границ фильтрации.
    """
    df_copy = df.copy()

    # Заменяет пропущенные значения меданной в столбце column, если fill_nan == True
    if fill_nan:
        df_copy[column] = nan_to_median(df_copy[column])
    if drop_nan_values:
        df_copy[column].dropna()

    #получение 25 и 75 процентилей
    q1 = df_copy[column].quantile(0.25)
    q3 = df_copy[column].quantile(0.75)

    # Вычисление рамок для фильтрации
    iqr = (q3 - q1) * multp
    low_bound = q1 - iqr
    up_bound = q3 + iqr
    outlines = 0 # индексы, значения строк столбца которых необходимо заменить медианой


    # Считаем за выбросы, которые заходят за up_bound или/и low_bound.
    if lower_bound and upper_bound:
        outlines = df_copy[(df_copy[column] < low_bound) | (df_copy[column] > up_bound)].index
    elif lower_bound:
        outlines = df_copy[df_copy[column] < low_bound].index
    elif upper_bound:
        outlines = df_copy[df_copy[column] > up_bound].index

    # Замена выбросов на np.nan
    df.loc[outlines, column] = np.nan
    return df


####################################################################################################


# Вторая функция: Ограничение и винзоризация для выбросов
def remove_outliers_and_handle_skewness(df, columns, threshold=1.5, cap_percentiles=(0.01, 0.99)):
    """
    Удаляет выбросы на основе асимметрии и применяет ограничение или преобразование.
    """
    df_cleaned = df.copy()

    for col in columns:
        # Обрабатываем асимметрию, применяя логарифмическое преобразование (если сильно асимметрично)
        if df[col].skew() > 1:
            df_cleaned[col] = np.log1p(df_cleaned[col])

        # Рассчитываем IQR для обнаружения выбросов
        Q1 = df_cleaned[col].quantile(0.25)
        Q3 = df_cleaned[col].quantile(0.75)
        IQR = Q3 - Q1

        # Рассчитываем нижний и верхний пределы для выбросов
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR

        # Применяем ограничение для выбросов по IQR и заменяем выбросы на медиану или ограниченные значения
        outliers = (df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)


        # Заменяем выбросы на NaN используя булевую маску
        df_cleaned.loc[outliers, col] = np.nan

        # В качестве альтернативы, применяем винзоризацию, ограничивая значения на указанных процентилях (если нужно более мягкое ограничение)
        lower_cap = df_cleaned[col].quantile(cap_percentiles[0])
        upper_cap = df_cleaned[col].quantile(cap_percentiles[1])
        df_cleaned[col] = df_cleaned[col].clip(lower=lower_cap, upper=upper_cap)

    return df_cleaned


####################################################################################################


# Гибридная функция, которая решает, применять ли ограничение или замену медианой в зависимости от асимметрии и степени выбросов
def hybrid_outlier_handling(df: pd.DataFrame, columns: list, threshold=1.5, multp=3, cap_percentiles=(0.01, 0.99)):
    """
    Гибридная функция для выбора между ограничением или заменой выбросов медианой,
    в зависимости от асимметрии столбца и степени выбросов.
    """
    df_cleaned = df.copy()

    for col in columns:
        # Обрабатываем асимметрию до обработки выбросов
        if df[col].skew() > 1:  # Если сильно асимметрично, сначала применим преобразование
            df_cleaned[col] = np.log1p(df_cleaned[col])

        # Рассчитываем IQR для обнаружения выбросов
        Q1 = df_cleaned[col].quantile(0.25)
        Q3 = df_cleaned[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR

        # Проверяем, есть ли экстремальные выбросы (за пределами 3x IQR)
        if any(df_cleaned[col] < (Q1 - 3 * IQR)) or any(df_cleaned[col] > (Q3 + 3 * IQR)):
            # Если есть экстремальные выбросы, заменяем их на медиану с помощью iqr_filter
            df_cleaned = radical_iqr_filter(df_cleaned, col, lower_bound=True, upper_bound=True, drop_nan_values=True,multp=multp)
        else:
            # В противном случае ограничиваем выбросы с помощью remove_outliers_and_handle_skewness
            df_cleaned = remove_outliers_and_handle_skewness(df_cleaned, [col], threshold=threshold, cap_percentiles=cap_percentiles)

    return df_cleaned


####################################################################################################


def super_train_test_split(df: pd.DataFrame, y: pd.Series):
    '''
    Делит данные на две выборки: 1. строки, значения необходимого нам столбца не имеют пропусков.
                                2. строки, значения необходимого нам столбца имеют пропуски.
    Каждый из этих пунктов так же делиться на две выборки: а) необходимый столбец.
                                                            б) остальные факторы.

    Аргументы:
        df: Pandas DataFrame, состоящий из факторов, инмеющих зависимость с признаком,
            в котором необходимо заполнить пропуски.

        y: Pandas Series, признак, пропуски которого необходимо заполнить.

    небольшой комментарий:
    У нас есть проблема - для заполенния пропусков с помощью какой-либо модели, необходимо,
    чтобы ВСЕ значения в других признаках были заполнены(не было пропусков).
    В противном случае модель ругается, что есть NaNы. Данный цикл устраняет данную проблему,
    временно заполняя пропуски в столбцах на медиану всех значений признака
    (кроме столбца, задача для которого изначально была заполнить пропуски с помощью модели).
    Дальше смотрите по комментариям
    '''
    X = df.copy()

    y_train = y[y.isnull() == False] # отбираем для тренировки те строки, в которых присутсвуют данные
    y_temp = y[y.isnull()] # просто мусор. Полезный

    idxs = y_temp.index # берём иднексы мусора(индексы,
                      # в строках которых есть пропуски, которые необходимо заполнить)
    X_train = X.drop(idxs) # делаем обучающую выборку из строк, в которых нет пропусков

    idxs = y_train.index # берём иднексы c изначально заполенными значениями
    X_test = X.drop(idxs) # отбрасываем строки с заполненными значениями в нужном нам столбце.
                        # Получается выборка с данными, на основе которых будут
                        # предсказываться пропущенные значения



    return X_train, X_test, y_train, y_temp


####################################################################################################


def split_for_grade(df: pd.DataFrame, target_column: pd.Series): # просто раздел данных на
                                                                 # выборки для обучения и тестирования
    X = df.copy()

    if target_column.name in X.columns:
        X.pop(target_column.name)
    y = target_column
    y1 = y[y.isnull() == False] # отбираем для тренировки те строки, в которых присутсвуют данные
    y_temp = y[y.isnull()] # просто мусор. Полезный

    idxs = y_temp.index # берём иднексы мусора(индексы,
                      # в строках которых есть пропуски, которые необходимо заполнить)
    X = X.drop(idxs) # делаем обучающую выборку из строк, в которых нет пропусков
    X = X.reset_index(drop=True)

    kernel = mf.ImputationKernel(
        data=X,
        random_state=42
    )
    y1 = y1.reset_index(drop=True)

    kernel.mice(iterations=1) # Количество итерация

    # Получаем датафрейм без пропусков
    X = kernel.complete_data()


    X_train, X_test, y_train, y_test = train_test_split(
        X, y1, test_size=0.2, random_state=42)
    return X_train.values, X_test.values, y_train.values, y_test.values

### **Классы**

In [33]:
class SBS():
    """
    Класс для последовательного обратного отбора признаков (Sequential Backward Selection).

    Алгоритм отбирает подмножество наиболее важных признаков,
    оптимизируя модель по метрикам качества (R-квадрат и MSE).

    Аргументы:
        estimator: Модель машинного обучения, которую нужно оптимизировать.
                   Должна поддерживать методы fit и predict.
        k_features: Целевое количество признаков для отбора.
        test_size: Доля данных для тестирования (кросс-валидация).
        random_state: Случайное зерно для воспроизводимости результатов.

    """
    def __init__(self, estimator, k_features,
                test_size=0.25, random_state=42):
        # Создаём копию модели, чтобы случайно не изменить исходную.
        self.estimator = clone(estimator)
        self.k_features = k_features
        self.test_size = test_size
        self.random_state = random_state

    def fit(self, X, y, own_split=False):
        """
        Обучает модель SBS и отбирает лучшие признаки.

        Аргументы:
            X: Матрица признаков.
            y: Вектор целевой переменной.
            own_split: Если True, использует пользовательскую функцию split_for_grade для разделения данных.
        """

        # Разделение данных на обучающую и тестовую выборки.
        X_train, X_test, y_train, y_test = split_for_grade(X, y)


        dim = X_train.shape[1]

        self.indices_ = list(range(dim))  # Индексы всех признаков.
        self.subsets_ = [self.indices_] # Список всех подмножеств признаков.

        # Вычисляем R-SQUARED и MSE.
        score = self._calc_score(X_train, y_train,
                                    X_test, y_test, self.indices_)
        self.scores_ = [score]

        while dim > self.k_features:
            scores = []
            subsets = []

            # Перебор всех возможных подмножеств с одним удаленным признаком.
            for p in combinations(self.indices_, r=dim - 1):
                score = self._calc_score(X_train, y_train,
                                            X_test, y_test, p)
                scores.append(score)
                subsets.append(p)

            # находим подмножества с лучшими значениями метрик
            best = np.argmax([i[0] for i in scores]) # Выбираем подмножество с наибольшим r2_score,
                                                     # т.к. данная метрика в приоритете. Так же отбор
                                                     # лучшей комбинации будет происходит вне класса.
            self.indices_ = subsets[best]
            self.subsets_.append(self.indices_)
            dim -= 1

            self.scores_.append(scores[best])
        self.k_score_ = self.scores_[-1]

        return self

    def _calc_score(self, X_train, y_train, X_test, y_test, indices):
        """
        Вычисляет метрики R-SQUARED и MSE для заданного подмножества признаков.

        Аргументы:
            X_train, y_train, X_test, y_test, indices: Данные для обучения и оценки модели.
        """
        self.estimator.fit(X_train[:, indices], y_train) # Обучение выбранной модели.

        y_pred = self.estimator.predict(X_test[:, indices]) # Предсказание значений.

        # Оценивание предсказаний метриками R2 и MSE
        score = [r2_score(y_test, y_pred), mean_absolute_percentage_error(y_test, y_pred)]
        return score


## Инициализация моделей и главного дата фрейма

инициализация моделей и методов

In [34]:
knn_model = KNeighborsRegressor(n_neighbors=3)
random_forest = RandomForestRegressor(random_state=42)
scaler = MinMaxScaler()

In [35]:
df_environmental_data = pd.read_csv("analysing_environmental_issues.csv", sep=',') # главный датафрейм

In [36]:
df = df_environmental_data.copy()

## предобработка перед оценкой моделей и предсказыванием значений

Перевод типов признаков в другие, нормализация, удаление дубликатов и обработка выбросов.

In [37]:
df.describe()

,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,stage_2_output_top_vacuum,...,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_danger_gas,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift
count,4159.000000,4177.000000,4180.000000,4209.000000,4223.000000,4169.000000,4218.000000,4226.000000,4196.000000,4205.000000,...,4226.000000,4170.000000,4231.000000,4174.000000,4156.000000,4159.000000,934.000000,4240.000000,4240.000000,4240.000000
mean,69.456160,86.674616,404.030844,79.215959,98.476441,56.764406,450.264177,261.478121,94.630858,59.045707,...,109.998318,42.777156,153.448811,20.162808,5.402151,313.779618,0.140139,22.438208,46.346776,1.483019
std,4.032077,31.155280,62.018933,3.027407,8.890578,7.858853,72.004423,43.201651,4.541636,11.180912,...,2.783694,4.472304,1.759867,3.080904,1.074238,104.519417,0.038566,1.243364,13.022949,0.499770
min,50.330000,19.950000,248.760000,66.130000,79.590000,34.070000,260.220000,134.920000,81.050000,33.150000,...,102.330000,25.940000,110.040000,-0.170000,2.350000,65.260000,0.020000,17.280000,0.710000,1.000000
25%,67.030000,64.820000,353.252500,77.430000,91.450000,52.250000,407.800000,230.027500,93.550000,49.910000,...,108.320000,40.960000,152.330000,18.100000,4.740000,245.160000,0.110000,21.680000,40.065000,1.000000
50%,70.030000,82.900000,389.395000,78.820000,97.200000,56.070000,436.960000,259.660000,95.530000,56.470000,...,109.265000,44.100000,153.210000,20.510000,5.500000,303.390000,0.140000,22.580000,47.870000,1.000000
75%,72.330000,105.570000,458.350000,80.850000,103.155000,60.030000,475.602500,290.480000,97.620000,68.780000,...,111.067500,45.860000,153.835000,22.130000,6.140000,366.005000,0.167500,23.280000,55.310000,2.000000
max,79.830000,233.370000,897.290000,105.460000,130.930000,125.360000,1000.750000,579.640000,109.900000,112.380000,...,123.500000,53.650000,157.680000,31.460000,7.980000,725.740000,0.340000,25.480000,107.050000,2.000000


Анализуря данные, выведенные в таблице выше, можно заметить болшие диапазоны значений, которые могут плохо повлиять на обучение модели и предсказание пропущенных значений, поэтому необходимо выполнить нормализацию данных.

известно, что столбец "DateTime" имеет неправильный тип данных, но его можно привести в числовой вид в диапазоне от 0 до 1, чтобы использовать в качестве фактора.

признак "stage_4_output_danger_gas" имеет большое количество пропущенных значений, поэтому он может плохо влиять на результаты и оценку метрик. Удалим его.

In [38]:
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')
time_diffs = df['DateTime'].diff().dt.total_seconds()
time_diffs = time_diffs.fillna(0)

# нормализуем даты из столбца DateTime
normalized_diffs = scaler.fit_transform(time_diffs.values.reshape(-1, 1)).flatten()

# вычисляет кумулятивную сумму элементов
normalized_times = np.cumsum(normalized_diffs)

# подставляем нормализованные значение
df['DateTime'] = normalized_times
#dateTime_series = df["DateTime"]
df.pop("stage_4_output_danger_gas") # значений мало, признак на данном этапе бесполезный.

df['work_shift'] = np.where(df['work_shift'] == 1.0, 0, 1)

df.head(5)

,DateTime,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,...,stage_3_input_steam,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift
0,0.000000,67.83,92.99,474.18,76.84,97.52,49.94,361.50,252.04,97.48,...,664.93,108.65,45.59,156.67,19.08,5.92,356.05,21.48,47.03,1
1,0.000413,67.83,91.82,473.68,76.15,97.82,48.55,354.75,244.87,97.66,...,671.68,108.71,45.89,156.76,19.15,5.94,357.69,21.48,45.05,1
2,0.000826,67.83,90.65,473.17,75.46,98.12,47.15,348.00,237.70,97.85,...,678.44,108.76,46.19,156.86,19.23,5.97,359.33,21.48,43.06,1
3,0.001239,67.93,90.24,473.59,75.26,97.79,49.33,356.74,249.87,97.50,...,717.99,108.63,45.87,156.41,19.36,5.97,339.99,21.48,46.01,1
4,0.001652,68.03,89.84,474.00,75.06,97.46,51.51,365.49,262.04,97.15,...,757.55,108.51,45.54,155.96,19.49,5.97,320.64,21.48,48.95,1


При поиске дубликатов было замечено, что все одинаковые строки данных, не считая признак DateTime, не содержат в себе данные, поэтому разумным действием будет их удаления

In [39]:
df[df.duplicated(subset=df.columns[1:])].head(10)

,DateTime,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,...,stage_3_input_steam,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift
1117,2.916976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1141,2.926890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1165,2.936803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1182,2.946716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1206,2.956629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1230,2.966543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1254,2.976456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1278,2.986369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1302,2.996283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1326,3.006196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [40]:
# Очень важно, что дубликаты будут найдены, если игнорировать столбец "DateTime".
df = df.drop_duplicates(subset=df.columns[1:], keep=False) # удаляем все дубликаты.

In [41]:
df.describe()

,DateTime,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,...,stage_3_input_steam,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift
count,4240.000000,4159.000000,4177.000000,4180.000000,4209.000000,4223.000000,4169.000000,4218.000000,4226.000000,4196.000000,...,4227.000000,4226.000000,4170.000000,4231.000000,4174.000000,4156.000000,4159.000000,4240.000000,4240.000000,4240.000000
mean,3.348423,69.456160,86.674616,404.030844,79.215959,98.476441,56.764406,450.264177,261.478121,94.630858,...,875.691462,109.998318,42.777156,153.448811,20.162808,5.402151,313.779618,22.438208,46.346776,0.483019
std,1.919090,4.032077,31.155280,62.018933,3.027407,8.890578,7.858853,72.004423,43.201651,4.541636,...,305.804871,2.783694,4.472304,1.759867,3.080904,1.074238,104.519417,1.243364,13.022949,0.499770
min,0.000000,50.330000,19.950000,248.760000,66.130000,79.590000,34.070000,260.220000,134.920000,81.050000,...,134.750000,102.330000,25.940000,110.040000,-0.170000,2.350000,65.260000,17.280000,0.710000,0.000000
25%,2.222945,67.030000,64.820000,353.252500,77.430000,91.450000,52.250000,407.800000,230.027500,93.550000,...,656.695000,108.320000,40.960000,152.330000,18.100000,4.740000,245.160000,21.680000,40.065000,0.000000
50%,3.644568,70.030000,82.900000,389.395000,78.820000,97.200000,56.070000,436.960000,259.660000,95.530000,...,844.150000,109.265000,44.100000,153.210000,20.510000,5.500000,303.390000,22.580000,47.870000,0.000000
75%,4.950537,72.330000,105.570000,458.350000,80.850000,103.155000,60.030000,475.602500,290.480000,97.620000,...,1144.395000,111.067500,45.860000,153.835000,22.130000,6.140000,366.005000,23.280000,55.310000,1.000000
max,7.749690,79.830000,233.370000,897.290000,105.460000,130.930000,125.360000,1000.750000,579.640000,109.900000,...,1616.930000,123.500000,53.650000,157.680000,31.460000,7.980000,725.740000,25.480000,107.050000,1.000000


In [42]:
df.isnull().any() # Проверка на пропуски.

,0
DateTime,False
stage_1_output_konv_avd,True
stage_2_input_water_sum,True
stage_2_output_bottom_pressure,True
stage_2_output_bottom_temp,True
stage_2_output_bottom_temp_hum_steam,True
stage_2_output_bottom_vacuum,True
stage_2_output_top_pressure,True
stage_2_output_top_pressure_at_end,True
stage_2_output_top_temp,True


Всего четыре признака не имеют пропусков в данных. Но после обработки выбросов данное число уменьшится до двух.

Было решено заменить выбросы на np.nan, так как замена на медиану или среднее значение может сильно исказить данные.

In [43]:
# Замена выбросов на np.nan.
df = hybrid_outlier_handling(df, df.columns[1:-1])

In [44]:
df.isnull().any() # Повторная проверка на пропуски в данных.

,0
DateTime,False
stage_1_output_konv_avd,True
stage_2_input_water_sum,True
stage_2_output_bottom_pressure,True
stage_2_output_bottom_temp,True
stage_2_output_bottom_temp_hum_steam,True
stage_2_output_bottom_vacuum,True
stage_2_output_top_pressure,True
stage_2_output_top_pressure_at_end,True
stage_2_output_top_temp,True


In [45]:
df.describe()

,DateTime,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,...,stage_3_input_steam,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift
count,4240.000000,4156.000000,4102.000000,4179.000000,4190.000000,4142.000000,4158.000000,4195.000000,4209.000000,4193.000000,...,4227.000000,4139.000000,4168.000000,4228.000000,4167.000000,4155.000000,3979.000000,4223.000000,4235.000000,4240.000000
mean,3.348423,69.469846,4.420832,403.912812,4.383318,97.993011,56.663100,6.097747,260.401780,94.633680,...,875.592890,4.707088,42.785214,153.466353,20.191613,5.399690,300.876557,22.456296,46.278789,0.483019
std,1.919090,4.001202,0.324346,61.555016,0.035164,8.202392,7.580481,0.137565,39.805043,4.527471,...,305.113641,0.019846,4.458217,1.587055,3.001165,1.062635,85.930415,1.197354,12.879112,0.499770
min,0.000000,52.630000,3.638649,248.760000,4.248781,84.704100,34.070000,5.565363,134.920000,81.350000,...,173.500000,4.637928,26.610000,148.950000,7.330000,3.025400,123.740000,19.580000,0.710000,0.000000
25%,2.222945,67.030000,4.197127,353.245000,4.362079,91.360000,52.240000,6.012896,229.960000,93.550000,...,656.695000,4.694188,40.960000,152.330000,18.105000,4.740000,242.990000,21.680000,40.050000,0.000000
50%,3.644568,70.030000,4.431888,389.380000,4.379524,96.980000,56.060000,6.081579,259.420000,95.530000,...,844.150000,4.702478,44.100000,153.210000,20.510000,5.500000,297.110000,22.580000,47.860000,0.000000
75%,4.950537,72.330000,4.667793,458.335000,4.404613,102.577500,59.987500,6.165554,290.120000,97.620000,...,1144.395000,4.717784,45.862500,153.840000,22.140000,6.140000,355.315000,23.280000,55.295000,1.000000
max,7.749690,79.830000,5.239361,764.760000,4.530231,117.563600,83.310000,6.624941,409.860000,109.300000,...,1555.389000,4.793225,53.650000,157.680000,31.460000,7.354600,525.304800,24.980000,98.750000,1.000000


## **SBS  для выбора факторов , на основе которых буддет обучаться модель и предсказываться значения**

Данный алгоритм выбирает лучшую комбинацию факторов для каждого столбца, которые имеют пропуски в данных, исходя из оценок KNN алгоритма с помощью метрик R2 и MAPE. Полученная комбинация записывается в словарь  **columns_dict_NaN_for_predict** с названием признака в качестве ключа

In [46]:
columns_dict_NaN_for_predict = {} # 'column_name': [best_columns_combination]

In [47]:
# запись изначальных индексов строк в лист и последующий сброс индексов в датфрейме.
# Это необходимое действие, так как mouseforest выдаёт ошибку,
# если нарушен порядок индексов, что и было сделано при удалении дубликатов.
index_list = df.index.tolist()
index_series = pd.Series(index_list)
df = df.reset_index(drop=True)

In [48]:
#инициализируем алгоритм sbs.

# Было решено использовать минимальное k_features = 2 в SBS, чтобы улавливать взаимодействия признаков,
# избегать потери информации, обеспечивать стабильность модели, сравнивать комбинации,
# и основываться на практике, где комбинации обычно лучше,
# а также избежать чрезмерного сокращения признаков на ранних итерациях алгоритма.
# Это позволяет искать комбинацию признаков, а не один единственный.
sbs = SBS(knn_model, k_features=2)

#поиск лучшей комбинации признаков для каждого столбца в датафрейме.
for column in df.columns:
    if df[column].isnull().any() == False:
        continue

    # подготовка данных
    y = df[column]
    X = df.copy()
    X.pop(column)

    #для корректной работы необходимо перевести названия столбцов в численный вид.
    new_names = [i for i in range(len(df.columns))]

    #скармливаем данные алгоритму sbs.
    sbs.fit(X, y)

    #переименовываем название факторов в численный вид
    X = X.rename(columns=dict(zip(X, new_names)))

    #инициализируем переменные для отбора лучшей комбинации признаков
    best_r2 = -1
    best_mape = float('inf')
    best_pair = None
    lk = -1



    #в sbs.scores_ записываются оценки метрик за все проверенные комбинации
    #поэтому необходимо отобрать лучшие показатели.
    #В ходе тестирований и анализа было замечено, что при лучших значениях метрик mse/mape,
    #метрика R2 показываля очень хорошие значения, поэтому из этого было принято отбирать пары
    #признаков исходя из значений метрики mape.
    for i, (r2_sc, mape_sc) in enumerate(sbs.scores_):
        #простой, но допустимы отбор
        if mape_sc < best_mape:
            best_r2 = r2_sc
            best_mape = mape_sc

            best_pair = [best_r2, best_mape]

            #создание списка индексов признаков.
            lk = list(sbs.subsets_[sbs.scores_.index([best_r2, best_mape])])

    #Так как при создании списка индексов признаков не учитывается, что был удалён
    #столбец, относительно которого ведутся вычесления, необходимо отредактировать
    #созданный массив.
    if df.columns.get_loc(y.name) in lk:
        index = lk.index(df.columns.get_loc(y.name))
        lk = np.array(lk)

        if index < len(lk) - 1:
            lk = np.concatenate((lk[:index], lk[index:] + 1))
        else:
            lk = lk[:index]
    else:
        for i in range(len(lk)):
            if lk[i] > df.columns.get_loc(y.name):
                lk[i] += 1

    #вывод результатов вычеслений.
    print(column)
    print(f"набор индексов лучших факторов: {lk}")
    #MAPE может выдавать ошибочные большие значения, но это не влияет сильно на итоговый результат
    print(f"Лучшая пара метрик: R2 = {best_pair[0]:.4f}, MAPE = {best_pair[1]:.4f}")
    print('=-----------------------------------------------')

    columns_dict_NaN_for_predict[column] = [col for col in list(df.columns[0:][lk]) if col != column]

#вывод итогового словаря
print(columns_dict_NaN_for_predict)


stage_1_output_konv_avd
набор индексов лучших факторов: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 19 20 21 22]
Лучшая пара метрик: R2 = 0.7154, MAPE = 0.0210
=-----------------------------------------------
stage_2_input_water_sum
набор индексов лучших факторов: [ 0  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]
Лучшая пара метрик: R2 = 0.8372, MAPE = 0.0155
=-----------------------------------------------
stage_2_output_bottom_pressure
набор индексов лучших факторов: [ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 17 18 19 20 21 22]
Лучшая пара метрик: R2 = 0.8315, MAPE = 0.0293
=-----------------------------------------------
stage_2_output_bottom_temp
набор индексов лучших факторов: [ 0  1  2  3  5  6  7  8  9 10 14 16 17 18 20 21]
Лучшая пара метрик: R2 = 0.9645, MAPE = 0.0009
=-----------------------------------------------
stage_2_output_bottom_temp_hum_steam
набор индексов лучших факторов: [ 0  1  2  3  4  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]
Луч

# Замена пропущенных значений

Для макисимально точного предсказания значений и качественного обучения модели было принято использовать две модели: KNN и RFR. Пропуски в данных, которые будут служить факторами для обучения и предсказания, заменяются на значения с MICE. Будет вестись запись метрик каждой модели для последующего нахождения лучшего варианта замены пропусков в каждом столбце.

Были использованы именно эти модели, потому что KNN показал свою эффективность в отборе факторов для обучения модели, а RFR является отличным вариантом для нахождения нелинейных зависимостей, обработки высокоразмерных данных.

In [49]:
#df

In [50]:
df1 = df.copy() # чек-поинт

In [51]:
df = df1.copy() # загрузка последнего сохранения

Создаём временный датафрейм **df_temp** и заполняем пропуски с помощью MICE. Данный датафрейм будет корректироваться после получения новых предсказаний модели.

In [52]:
df_temp = df.copy()
kernel = mf.ImputationKernel(
    data=df_temp,
    random_state=42
)

kernel.mice(iterations=5)

# Получаем временный датафрейм без пропусков
df_temp = kernel.complete_data()

In [53]:
#создаём временную копию главного датафрейма.
df_without_nan_by_cnn = df.copy()

#словарь со значениями метрики r2 каждого столбца,
#в котором обнаружены пропуски
metrics_by_cnn = {}

In [54]:
for column in columns_dict_NaN_for_predict.keys(): # для каждого столбца в котором остались пропущенные значения

    X = df_temp.loc[:, columns_dict_NaN_for_predict[column]].copy() # все зависимые признаки с столбцом column ()
    y = df_without_nan_by_cnn[column] # наш столбец.


    y_for_grade = y[y.isnull() == False] # отбираем для тренировки те строки, в которых присутсвуют данные
    y_temp = y[y.isnull()] # просто мусор. Полезный

    idxs = y_temp.index # берём иднексы мусора(индексы,
                      # в строках которых есть пропуски, которые необходимо заполнить)
    X_for_grade = X.drop(idxs) # делаем обучающую выборку из строк, в которых нет пропусков

    X_train, X_test, y_train, y_temp = train_test_split(X_for_grade, y_for_grade, test_size=0.2, random_state=42) # Делим наши данные на
                                                                    # на обучающую и тестовую выборку
    knn_model.fit(X_train, y_train) # обучаем модель knn(для каждого столбца)
    y_pred = knn_model.predict(X_test) # предсказываем пропущенные значения

    #оценка модели
    r2 = r2_score(y_temp, y_pred)
    mse = mean_squared_error(y_temp, y_pred)

    metrics_by_cnn[column] = [r2] #запись оценку метрики r2 в словарь

    print("-" * 50)
    print(f'{column} -> r2: {r2}, mse: {mse}')


    X_train, X_test, y_train, y_temp = super_train_test_split(X, y) # Делим наши данные на
                                                                    # на обучающую и тестовую выборку
    knn_model.fit(X_train, y_train) # обучаем модель knn(для каждого столбца)

    y_pred = knn_model.predict(X_test) # предсказываем пропущенные значения

    df_without_nan_by_cnn.loc[X_test.index, column] = y_pred # вставляем предсказания на места пропущенных значений
    df_temp.loc[X_test.index, column] = y_pred # вставляем предсказания на места пропущенных значений для более точных предсказаний

--------------------------------------------------
stage_1_output_konv_avd -> r2: 0.6999711648805047, mse: 4.529509882478633
--------------------------------------------------
stage_2_input_water_sum -> r2: 0.8432823162961598, mse: 0.01716232638451944
--------------------------------------------------
stage_2_output_bottom_pressure -> r2: 0.8226586782684103, mse: 715.1824887426901
--------------------------------------------------
stage_2_output_bottom_temp -> r2: 0.9652891265083556, mse: 4.4238206965517145e-05
--------------------------------------------------
stage_2_output_bottom_temp_hum_steam -> r2: 0.7374588929034909, mse: 16.65776044724835
--------------------------------------------------
stage_2_output_bottom_vacuum -> r2: 0.9454255504572455, mse: 3.0590371661324807
--------------------------------------------------
stage_2_output_top_pressure -> r2: 0.8369826103018789, mse: 0.0029703208061332537
--------------------------------------------------
stage_2_output_top_pressure_at

In [55]:
df = df1.copy() # загрузка последнего сохранения

In [56]:
#аналогичные действия были описаны выше
df_temp = df.copy()
kernel = mf.ImputationKernel(data=df_temp, random_state=42)
kernel.mice(iterations=5)
df_temp = kernel.complete_data()

In [57]:
#создаём временную копию главного датафрейма.
df_without_nan_by_rfr = df.copy()

#словарь со значениями метрики r2 каждого столбца,
#в котором обнаружены пропуски
metrics_by_rfr = {}

In [58]:
for column in columns_dict_NaN_for_predict.keys(): # для каждого столбца в котором остались пропущенные значения
    y = df_without_nan_by_rfr[column] # наш столбец.
    X = df_temp.loc[:, columns_dict_NaN_for_predict[column]].copy()

    y_for_grade = y[y.isnull() == False] # отбираем для тренировки те строки, в которых присутсвуют данные
    y_temp = y[y.isnull()] # просто мусор. Полезный

    idxs = y_temp.index # берём иднексы мусора(индексы,
                      # в строках которых есть пропуски, которые необходимо заполнить)
    X_for_grade = X.drop(idxs) # делаем обучающую выборку из строк, в которых нет пропусков

    X_train, X_test, y_train, y_temp = train_test_split(X_for_grade, y_for_grade, test_size=0.2, random_state=42) # Делим наши данные на
                                                                    # на обучающую и тестовую выборку

    random_forest.fit(X_train, y_train) # обучаем модель Forest Regression(для каждого столбца)
    y_pred = random_forest.predict(X_test) # предсказываем пропущенные значения

    #оценка модели
    r2 = r2_score(y_temp, y_pred)
    mse = mean_squared_error(y_temp, y_pred)

    metrics_by_rfr[column] = [r2] #запись оценку метрики r2 в словарь

    print("-" * 50)
    print(f'{column} -> r2: {r2}, mse: {mse}')


    # Делим наши данные на обучающую и тестовую выборку.
    X_train, X_test, y_train, y_temp = super_train_test_split(X, y) # Делим наши данные на
                                                                    # на обучающую и тестовую выборку

    random_forest.fit(X_train, y_train) # обучаем модель Random Forest Regression(для каждого столбца)

    y_pred = random_forest.predict(X_test) # предсказываем пропущенные значения

    df_without_nan_by_rfr.loc[X_test.index, column] = y_pred # вставляем предсказания на места пропущенных значений
    df_temp.loc[X_test.index, column] = y_pred





--------------------------------------------------
stage_1_output_konv_avd -> r2: 0.8490965841318656, mse: 2.2781760733173075
--------------------------------------------------
stage_2_input_water_sum -> r2: 0.9475134461819731, mse: 0.005747860395422307
--------------------------------------------------
stage_2_output_bottom_pressure -> r2: 0.9335949765602988, mse: 267.79833072690224
--------------------------------------------------
stage_2_output_bottom_temp -> r2: 0.9782501497385349, mse: 2.771968206352737e-05
--------------------------------------------------
stage_2_output_bottom_temp_hum_steam -> r2: 0.9480680848555818, mse: 3.294986494151372
--------------------------------------------------
stage_2_output_bottom_vacuum -> r2: 0.9698992152550984, mse: 1.6872257995432691
--------------------------------------------------
stage_2_output_top_pressure -> r2: 0.9539347468400295, mse: 0.000839349594262538
--------------------------------------------------
stage_2_output_top_pressure_a

In [59]:
# заменяем столбец с пропущенными данными лучшим, опираясь на метрику r2
for column in df.columns[1:-1]: # первые два столцба не имеют пропусков
    if metrics_by_rfr[column] > metrics_by_cnn[column]:
        df[column] = df_without_nan_by_rfr[column]
    else:
        df[column] = df_without_nan_by_cnn[column]



# Сохранение обработанного датафрейма

In [60]:
df = df.set_index(index_series) # заменяем сброшенные индесы на старые для корректной работы.

In [61]:
# добавляем столб stage_4_output_danger_gas
df["stage_4_output_danger_gas"] = df_environmental_data['stage_4_output_danger_gas'].loc[df.index]

In [62]:
df.describe()

,DateTime,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,...,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift,stage_4_output_danger_gas
count,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,...,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,4240.000000,934.000000
mean,3.348423,69.469334,4.420206,403.941219,4.383774,98.267039,56.703319,6.100141,260.422384,94.622648,...,4.708227,42.783293,153.464613,20.186574,5.399413,305.397236,22.451004,46.328325,0.483019,0.140139
std,1.919090,3.985938,0.330785,61.456774,0.036012,8.358436,7.615988,0.141162,39.721820,4.538063,...,0.021330,4.451105,1.585360,3.000550,1.060880,87.528102,1.197935,12.952107,0.499770,0.038566
min,0.000000,52.630000,3.638649,248.760000,4.248781,84.704100,34.070000,5.565363,134.920000,81.350000,...,4.637928,26.610000,148.950000,7.330000,3.025400,123.740000,19.580000,0.710000,0.000000,0.020000
25%,2.222945,67.030000,4.187569,353.252500,4.362207,91.450000,52.240000,6.013189,230.072500,93.530000,...,4.694279,40.960000,152.337500,18.100000,4.740000,245.477500,21.680000,40.065000,0.000000,0.110000
50%,3.644568,70.030000,4.428970,389.485000,4.379774,97.200000,56.070000,6.082128,259.375000,95.530000,...,4.702932,44.105000,153.210000,20.510000,5.500000,303.020000,22.580000,47.870000,0.000000,0.140000
75%,4.950537,72.330000,4.670209,458.350000,4.404797,103.152500,60.022500,6.167013,290.000000,97.620000,...,4.719034,45.860000,153.830000,22.132500,6.140000,360.993232,23.280000,55.310000,1.000000,0.167500
max,7.749690,79.830000,5.239361,764.760000,4.530231,117.563600,83.310000,6.624941,409.860000,109.300000,...,4.793225,53.650000,157.680000,31.460000,7.354600,525.304800,24.980000,98.750000,1.000000,0.340000


In [63]:
df.to_csv("data_imputed_stage4gas_unfilled.csv", encoding='utf-8', index=False)